![alt text](http://www.charityscience.com/uploads/1/0/7/2/10726656/editor/black-vertical_1.png?1493840138 "Logo Title Text 1")
## Data Cleaning Exercise for Internship



The data offered mimics the type of data that would be cleaned from the SMS Vaccination reminder program at charity science health. It is raw data that needs to be first cleaned and second used to generate a new variable based on the lateness of vaccinations

The first part of the task is to import the data from the raw file, drop all variables which have completely missing observations, generate a childid and reshape the data into a long format

In [56]:
import delimited "testdata.csv", varnames(1) clear

foreach var of varlist _all {
     capture assert mi(`var')
     if !_rc {
        drop `var'
     }
}


gen childid = _n
reshape long vaccine_name@ vaccine_date@, i(childid) j(vaccinenumber)




(17 vars, 25 obs)



(note: j = 1 2 3)

Data                               wide   ->   long
-----------------------------------------------------------------------------
Number of obs.                       25   ->      75
Number of variables                  14   ->      11
j variable (3 values)                     ->   vaccinenumber
xij variables:
vaccine_name1 vaccine_name2 vaccine_name3 ->   vaccine_name
vaccine_date1 vaccine_date2 vaccine_date3 ->   vaccine_date
-----------------------------------------------------------------------------


The next part of the task is to format the birth and vaccination dates of the children in such a way that the late variable can be generated. The data originally gives dates in format "01-JAN-19", STATA is very well equipped to deal with a format like this however there is a very big problem. 19 does not correspond to any particular year after 1960 (the base year for STATA) and therefore a trick needs to be used to insert the missing digits and to run the date() function.

In [57]:
foreach v of varlist child_dateofbirth vaccine_date {

split `v', parse(-)
gen `v'm = month(date(`v'2, "M"))
destring `v'1 `v'm `v'3, replace
replace `v'3 = `v'3+2000
gen `v'date = mdy(`v'm, `v'1, `v'3)
drop `v'1 `v'm `v'3 `v'2
}

drop child_dateofbirth vaccine_date

sort childid vaccine_datedate
by childid: replace vaccinenumber = _n

rename vaccine_datedate vaccine_date
rename child_dateofbirth childdob


variables created as string: 
child_date~1  child_date~2  child_date~3
child_dateofbirth1: all characters numeric; replaced as byte
child_dateofbirthm already numeric; no replace
child_dateofbirth3: all characters numeric; replaced as byte
variable child_dateofbirth3 was byte now int
(75 real changes made)
variables created as string: 
vaccine_da~1  vaccine_da~2  vaccine_da~3
(31 missing values generated)
vaccine_date1: all characters numeric; replaced as byte
(31 missing values generated)
vaccine_datem already numeric; no replace
vaccine_date3: all characters numeric; replaced as byte
(31 missing values generated)
variable vaccine_date3 was byte now int
(44 real changes made)
(31 missing values generated)



(27 real changes made)




The next part of the task was to generate the late times based on the vaccination schedule provided.
1. BCG - to be given at birth
2. Hepatitis B - to be given at birth
3. Penta1 - 6 weeks after birth
4. Penta2 - 4 weeks after Penta1
5. Penta3 - 4 weeks after Penta2

My code used the strpos() function to determine which vaccine was specified if strpos(vaccine_name#, "vaccine"). The date of PENTA2 was relative so for the 2nd and 3rd vaccination I calculated the due date from the previous vaccination. I do not assume that the PENTA vaccines were recorded accurately hence I use two lines of code to distinguish whether or not PENTA2 or PENTA3 was given in between another vaccination.


In [58]:

reshape wide vaccine_name vaccine_date, i(childid) j(vaccinenumber)

gen vaccine_late1 = vaccine_date1 - childdob if strpos(vaccine_name1, "BCG") | strpos(vaccine_name1, "Hepatitis B Birth Dose")> 0
gen vaccine_late2 = vaccine_date2 - childdob if strpos(vaccine_name1, "BCG") | strpos(vaccine_name1, "Hepatitis B Birth Dose")> 0
replace vaccine_late2 = vaccine_date2 - 42 - childdob if strpos(vaccine_name2, "PENTA1") 
replace vaccine_late2 = vaccine_date2 - 28 - vaccine_date1 if strpos(vaccine_name2, "PENTA2") & strpos(vaccine_name1, "PENTA1")> 0 
replace vaccine_late2 = vaccine_date2 - 28 - childdob if strpos(vaccine_name2, "PENTA3") & strpos(vaccine_name1, "PENTA2")>0

gen vaccine_late3 = vaccine_date3 - childdob if strpos(vaccine_name3, "BCG") | strpos(vaccine_name3, "Hepatitis B Birth Dose")> 0
replace vaccine_late3 = vaccine_date3 - 42 - vaccine_date1 if strpos(vaccine_name3, "PENTA1")
replace vaccine_late3 = vaccine_date3 - 28 - vaccine_date1 if strpos(vaccine_name3, "PENTA2") & strpos(vaccine_name1, "PENTA1")> 0 
replace vaccine_late3 = vaccine_date3 - 28 - vaccine_date2 if strpos(vaccine_name3, "PENTA2") & strpos(vaccine_name2, "PENTA1")> 0 
replace vaccine_late3 = vaccine_date3 - 28 - vaccine_date1 if strpos(vaccine_name3, "PENTA3") & strpos(vaccine_name1, "PENTA2")> 0 
replace vaccine_late3 = vaccine_date3 - 28 - vaccine_date2 if strpos(vaccine_name3, "PENTA3") & strpos(vaccine_name2, "PENTA2")> 0 



(note: j = 1 2 3)

Data                               long   ->   wide
-----------------------------------------------------------------------------
Number of obs.                       75   ->      25
Number of variables                  11   ->      14
j variable (3 values)     vaccinenumber   ->   (dropped)
xij variables:
                           vaccine_name   ->   vaccine_name1 vaccine_name2 vaccine_name3
                           vaccine_date   ->   vaccine_date1 vaccine_date2 vaccine_date3
-----------------------------------------------------------------------------

(10 missing values generated)

(14 missing values generated)

(3 real changes made)

(4 real changes made)

(1 real change made)

(25 missing values generated)

(2 real changes made)

(1 real change made)

(1 real change made)

(0 real changes made)

(0 real changes made)


Now that all the data has generated its summary statistics we clean up the data to make it more useable for further analysis

In [59]:
foreach v of varlist childdob vaccine_date1 vaccine_date2 vaccine_date3 {

format `v' %td
}

order childid childdob vaccine_name1 vaccine_date1 vaccine_late1 vaccine_name2 vaccine_date2  vaccine_late2 vaccine_name3 vaccine_date3  vaccine_late3

export delimited "vaccine_overdue.csv", delimiter(",") replace




file vaccine_overdue.csv saved


Below is the completed result

In [60]:
%browse

,childid,childdob,vaccine_name1,vaccine_date1,vaccine_late1,vaccine_name2,vaccine_date2,vaccine_late2,vaccine_name3,vaccine_date3,vaccine_late3,vaccinationsgiven,vaccine1_ifgiven,vaccine2_ifgiven,vaccine3_ifgiven,vaccine4_ifgiven,vaccine5_ifgiven
1,1,18feb2019,BCG,18feb2019,0,.,.,.,.,.,.,4,0,0,0,1,0
2,2,14feb2019,BCG,15feb2019,1,.,.,.,.,.,.,4,0,0,0,1,0
3,3,13jan2019,BCG,14jan2019,1,.,.,.,.,.,.,4,0,0,0,1,0
4,4,17jan2019,.,.,.,.,.,.,.,.,.,-9999,0,0,0,0,0
5,5,04jan2019,BCG,06feb2019,33,.,.,.,.,.,.,4,0,0,0,1,0
6,6,24dec2018,Hepatitis B Birth Dose,26dec2018,2,BCG,26dec2018,2,PENTA1,13feb2019,7,1 4 5,1,0,0,1,1
7,7,26dec2018,BCG,27dec2018,1,Hepatitis B Birth Dose,27dec2018,1,.,.,.,4 5,0,0,0,1,1
8,8,22nov2018,BCG,23dec2018,31,Hepatitis B Birth Dose,23dec2018,31,.,.,.,4 5,1,0,0,1,1
9,9,21dec2017,PENTA1,23apr2018,.,BCG,23apr2018,.,PENTA2,21aug2018,92,1 2 4,1,1,0,1,0
10,10,12dec2018,Hepatitis B Birth Dose,13dec2018,1,BCG,13dec2018,1,.,.,.,4 5,0,0,0,1,1
